In [1]:
from docx import *
from docx.shared import RGBColor
from string import ascii_uppercase
import pandas as pd
import re

In [2]:
def toStr(s):  
    str = " " 
    return (str.join(s))

In [3]:
def clean(x):
    # hilangkan digit 
    temp1 = re.sub(r"\d","", x)

    # ilangkan tab 
    temp2 = re.sub(r"\t","", temp1)

    # hilangkan spasi,tab, newline diakhir dan koma
    result = re.sub(r"\s+$|,|;", "", temp2)
    
    return result

In [4]:
def cleanEnd(x):
    x = re.sub(r"[^a-zA-Z\-\s]", '', x) # menghilangkan karakter selain huruf abjad dan '-'
    x = re.sub(' +', ' ',x).strip() # menghilangkan spasi yang berulang
    x = re.sub("\\s*-\\s*", "-",x).strip() # menghilangkan spasi setelah atau sebelum tanda '-'
    
    return x

In [5]:
def insertData(utama,turunan,gabungan,similiar,kelas,panah,keterangan):
    x = [utama,turunan,gabungan,similiar,kelas,panah,keterangan]
    # print(x)
    y = pd.DataFrame(data=[x],columns=['Kata Dasar','Kata Turunan','Kata Gabungan','Kata Similar','Kelas Kata','Arah Panah','Keterangan'])
    
    return x,y

In [6]:
def ambilKata(word):
    # jika ada tanda () merupakan similiar
    if '(' in word :
        x, y = re.sub(r"[/(].*?[/))=]","",word),re.sub(r"[()=]","",word)
        x, y = re.sub(r"\s$","",x),re.sub(r"\s$","",y)
        utama = x
        similar = y
        tempUtama = utama
                    
    else :
        word = re.sub(r"[()]|\s$|-\s","",word)
        utama = word
        similar = ""
        tempUtama = utama

    return utama,similar,tempUtama

In [7]:
for upper in ascii_uppercase:
    document = Document("Prepared/KBBI-"+upper+"-done.docx")

    korpus = pd.DataFrame(columns=['Kata Dasar','Kata Turunan','Kata Gabungan','Kata Similar','Kelas Kata','Arah Panah','Keterangan'])

    kelasKata = ['n','v','a','adv','num','p','pron']
    ket = ["Ab","Abr","Ach","Adm","Ag","akr","Anat","Antr","Ar","Arg","ark","Astron","Astr","Astrol","Bg","Bjr","Bio","Biol","Bl","Bld","Bot","Bt","Btk","Brk","cak","Cn","Dag","Dik","Dok","Dn","Dy","Ek","Ekg","El","Ent","Far","Fifol","Fil","Filol","Fils","Fis","Fir","Geo","Geog","Geol","Gra","Graf","Gy","Hd","hid","Hin","Hind","hor","Huk","Hut","Isl","Ikn","Ing","Isi","Isl","Jak","Jb","Jk","Jp","Jw","Jyw","k","Kal","Kap","kas","Kat","kep","Kes","Keu","Kim","kl","kp","Kris","Kom","Komp","lesl","Lay","Ling","Lis","lt","Man","Mat","Md","Mdr","Mek","Mes","Met","Mil","Mk","Mks","Mn","Mu","Mus","ok","Olr","on","Opt","Org","Orl","Pem","Pet","Plb","Pol","Psi","Pr","Prot","Publ","Sas","sb","sd","sel","sj","Sen","Sos","Skr","Skt","Sng","Sos","Stat","Tan","tbl","tas","Tek","terb","Terb","Tern","tld","Tns","Us","zat","Zool"]
    kosong = ["", " " , "  "]
    tempTurunan =""
    tempUtama = ""

    for paragraph in document.paragraphs:

        utama = ""
        turunan = ""
        gabungan = ""
        similar = ""
        kelas = ""
        panah = ""
        keterangan = ""

        tempKelas = []
        tempKeterangan = []
    #     tempUtama = ""

        tandaPanah = False
        index = 0
        for run in paragraph.runs:


            # teks warna merah (sublema)
            if ((run.font.color.rgb == RGBColor(255,0,0))):
    #             print(run.text+","+str(index))

                # sublema
                if(run.bold):

                    run.text = clean(run.text)

                    if run.text not in kosong and tandaPanah != True:

                        # tanda panah di sub lema
                        if (re.search("‹", run.text)) :
    #                         print(run.text)
                            if re.search("\d", paragraph.runs[index+2].text):
                                tempPanah = paragraph.runs[index+3].text
    #                             panah = tempPanah
                            else :
                                tempPanah = paragraph.runs[index+2].text
    #                             panah = tempPanah

                            result = ambilKata(tempPanah)

                            if result[1] != "":
                                panah = result[0]
                                x = insertData(utama,turunan,gabungan,similar,kelas,panah,keterangan)
                                print(x[0])
                                korpus  = korpus.append([x[1]],ignore_index=True)

                                panah = result[1]

                            else :

                                panah = result[0]

                            tandaPanah = True

                        else:

                            # sublema
                            if utama != "" or turunan != "" or kelas != "" :

                                # insert
                                x = insertData(utama,turunan,gabungan,similar,kelas,panah,keterangan)
                                print(x[0])
                                korpus  = korpus.append([x[1]],ignore_index=True)
                                utama = ""

                            # menangani jika -- ga bold
                            if index == 1 and re.search("--", paragraph.runs[index-1].text):

                                run.text = tempUtama+" "+run.text

                                result = ambilKata(run.text)

                                gabungan     = result[0]
                                similar  = result[1]
                                kelas = ""
                                keterangan = ""
                                tempKeterangan = []
                                kelas = ""

                            # menangani jika ~ ga bold
                            elif index == 1 and re.search("~", paragraph.runs[index-1].text):
        #                         print(paragraph.runs[index+1].text)
        #                         print(run.text)
                                run.text = tempTurunan+" "+run.text

                                result = ambilKata(run.text)

                                gabungan     = result[0]
                                similar  = result[1]
                                kelas = ""
                                keterangan = ""

                            else:
                                if re.search("--", run.text):
                                    # ganti -- dengan lema (sebelumnya/tempUtama)
                                    run.text = re.sub(r"--",tempUtama,run.text)

                                    result = ambilKata(run.text)

                                    gabungan     = result[0]
                                    similar  = result[1]
                                    kelas = ""
                                    keterangan = ""
                                    tempKeterangan = []

                                elif re.search("~", run.text):
        #                             print(run.text)
                                    # ganti ~ dengan sublema (sebelumnya/tempTurunan)
                                    run.text = re.sub(r"~",tempTurunan,run.text)

                                    result = ambilKata(run.text)

                                    gabungan     = result[0]
                                    similar  = result[1]
                                    kelas = ""
                                    keterangan = ""
                                    tempKeterangan = []

                                else:
                                    result = ambilKata(run.text)

                                    turunan     = result[0]
                                    similar    = result[1]
                                    tempTurunan = result[2]
                                    kelas = ""
                                    keterangan = ""
                                    tempKeterangan = []


                elif(run.italic):

                    run.text = clean(run.text)

                    x = run.text.split(" ")

                    # keterangan
                    for y in x:

                        if (y in kelasKata or y in ket) and y not in tempKeterangan:
                            tempKeterangan.append(y)

                    if tempKeterangan != "":
                        for temp in tempKeterangan :
                            if temp in ket:
                                keterangan = " ".join(tempKeterangan)

                    # kelas kata
                    for y in x:

                        if y in kelasKata and y != kelas:
                            if kelas != "":
                                # insert
                                x = insertData(utama,turunan,gabungan,similar,kelas,panah,keterangan)
                                print(x[0])
                                korpus  = korpus.append([x[1]],ignore_index=True)
                            kelas = y  

            # teks warna hijau (similar)
            elif ((run.font.color.rgb == RGBColor(0,176,80))):
    #             print(run.text)
                if(run.bold):

                    run.text = clean(run.text)

                    if run.text not in kosong and re.search("\w", run.text) :

                        similar = re.sub(r"[()=]","",run.text)

    #                     print(tempUtama)
                        if re.search("--", similar):
                            # ganti -- dengan lema (sebelumnya/tempUtama)
                            similar = re.sub(r"--",tempUtama,similar)

                        elif re.search("~", similar):
                            # ganti -- dengan lema (sebelumnya/tempUtama)
                            similar = re.sub(r"~",tempTurunan,similar)

    #                     print(similar)

            # teks bukan warna merah (lema)
            else:

                # lema
                if(run.bold):
    #                 print(run.text)
                    run.text = clean(run.text)

    #                 if run.text not in kosong and re.search("\w", run.text) :

                    if run.text not in kosong and tandaPanah != True:

                        # tanda panah di lema
                        temp = []
                        if (re.search("‹", run.text)) :
    #                         print(run.text)
                            if re.search("\d", paragraph.runs[index+2].text):
                                tempPanah = paragraph.runs[index+3].text
    #                             panah = tempPanah
                            else :
                                tempPanah = paragraph.runs[index+2].text
    #                             panah = tempPanah

                            result = ambilKata(tempPanah)

                            if result[1] != "":
                                panah = result[0]
                                x = insertData(utama,turunan,gabungan,similar,kelas,panah,keterangan)
                                print(x[0])
                                korpus  = korpus.append([x[1]],ignore_index=True)

                                panah = result[1]

                            else :

                                panah = result[0]

    #                         tempUtama = paragraph.runs[index-1].text
    #                         utama = tempUtama
                            tandaPanah = True

                        else :

                            # lema
                            if utama != "" or turunan!= "" or kelas != "":
                                # insert
                                x = insertData(utama,turunan,gabungan,similar,kelas,panah,keterangan)
                                print(x[0])
                                korpus  = korpus.append([x[1]],ignore_index=True)

                                turunan = ""

                            result = ambilKata(run.text)

                            utama     = result[0]
                            similar  = result[1]
                            tempUtama = result[2]
                            kelas = ""
                            keterangan = ""
                            tempKeterangan = []

    #                     if utama != "" or turunan!= "" or kelas != "":
    #                         # insert
    #                         x = insertData(utama,turunan,gabungan,similar,kelas,panah,keterangan)
    #                         print(x[0])
    #                         korpus  = korpus.append([x[1]],ignore_index=True)

    #                         turunan = ""

    #                     result = ambilKata(run.text)

    #                     utama     = result[0]
    #                     similar  = result[1]
    #                     tempUtama = result[2]
    #                     kelas = ""
    #                     keterangan = ""
    #                     tempKeterangan = []

                elif(run.italic):

                    run.text = clean(run.text)

                    x = run.text.split(" ")

                    # keterangan
                    for y in x:

                        if (y in kelasKata or y in ket) and y not in tempKeterangan:
                            tempKeterangan.append(y)

                    if tempKeterangan != "":
                        for temp in tempKeterangan :
                            if temp in ket:
                                keterangan = " ".join(tempKeterangan)

                    # kelas kata
                    for y in x:

                        if y in kelasKata and y != kelas:
                            if kelas != "":
                                # insert
                                x = insertData(utama,turunan,gabungan,similar,kelas,panah,keterangan)
                                print(x[0])
                                korpus  = korpus.append([x[1]],ignore_index=True)
                            kelas = y

                # tanda panah yang heading
    #             else:
    #                 run.text = clean(run.text)
    # #                 print(run.text)

    #                 if run.text not in kosong :

    #                     # tanda panah
    #                     temp = []
    #                     if (re.search("‹", run.text)) :

    #                         if re.search("\d", paragraph.runs[index+2].text):
    #                             tempPanah = paragraph.runs[index+3].text
    #                             panah = tempPanah
    #                         else :
    #                             tempPanah = paragraph.runs[index+2].text
    #                             panah = tempPanah

    #                         tempUtama = paragraph.runs[index-1].text
    #                         utama = tempUtama

            index += 1

        # insert
        if utama not in kosong or turunan not in kosong or gabungan not in kosong or similar not in kosong or kelas not in kosong or panah not in kosong or keterangan not in kosong :
            x = insertData(cleanEnd(utama),cleanEnd(turunan),cleanEnd(gabungan),cleanEnd(similar),cleanEnd(kelas),cleanEnd(panah),cleanEnd(keterangan))
            print(x[0])
            korpus  = korpus.append([x[1]],ignore_index=True)
        
    korpus.to_excel (r"Done/Korpus-"+upper+".xlsx", index = False, header=True)

['A', '', '', 'a', 'n', '', '']
['A', '', '', '', 'n', '', '']
['a', '', '', '', 'n', '', '']
['ab', '', '', '', 'n', '', 'ark n']
['aba', '', '', '', 'n', '', '']
['aba-aba', '', '', '', 'n', '', '']
['abad', '', '', '', 'n', '', '']
['', '', 'abad kekal', '', '', '', '']
['', '', 'abad keemasan', '', '', '', '']
['', '', 'abad pertengahan', '', '', '', '']
['', 'berabad-abad', '', '', 'v', '', '']
['', 'seabad', '', '', 'n', '', '']
['abadi', '', '', '', 'a', '', '']
['', 'mengabadi', '', '', 'v', '', '']
['', 'mengabadikan', '', '', 'v', '', '']
['', 'pengabadian', '', '', 'n', '', '']
['', 'keabadian', '', '', 'n', '', '']
['abadiah', '', '', '', 'n', '', 'Ar n']
['abadiat', '', '', '', '', 'abadiah', '']
['abah', '', '', '', 'n', '', '']
['', 'mengabah', '', '', 'v', '', '']
['', 'mengabahkan', '', '', 'v', '', '']
['abah', '', '', '', '', 'aba', '']
['abah-abah', '', '', '', 'n', '', '']
['', '', 'abah-abah kuda', '', '', '', '']
['', '', 'abah-abah perahu', '', '', '', '']
['', 

['', 'mengacu', '', '', 'v', '', '']
['', 'mengacukan', '', '', 'v', '', '']
['acu', '', '', '', 'v', '', '']
['', 'mengacu mencetak', '', '', 'v', '', '']
['', 'acuan', '', '', 'n', '', '']
['', '', 'acuan sepatu', '', '', '', '']
['', '', 'acuan tegangan', '', 'n', '', '']
['acuh', '', '', '', 'a', '', '']
['', '', 'acuh tak acuh', '', '', '', '']
['', 'acuhan', '', '', 'n', '', '']
['', 'mengacuhkan', '', '', 'v', '', '']
['acum', '', '', '', 'v', '', '']
['', 'mengacumkan', '', '', 'v', '', '']
['', 'acuman', '', '', 'n', '', '']
['', 'pengacum', '', '', 'n', '', '']
['acung', '', '', '', 'v', '', '']
['', 'mengacung', '', '', 'v', '', '']
['', 'mengacungkan', '', '', 'v', '', '']
['acung', '', '', '', 'v', '', '']
['', 'mengacung', '', '', 'v', '', '']
['ad interim', '', '', '', 'a', '', '']
['ada', '', '', '', 'v', '', '']
['', 'adalah', '', '', 'v', '', '']
['', 'adanya', '', '', 'n', '', '']
['', 'berada', '', '', 'v', '', '']
['', 'berada-ada', '', '', 'v', '', '']
['', 'menga

['', '', 'aerodinamika supersonik', '', 'n', '', '']
['aerofon', '', '', '', 'n', '', 'n Mus']
['aerograf', '', '', '', 'n', '', 'n Fis']
['aerogram', '', '', '', 'n', '', '']
['aerolit', '', '', '', 'n', '', 'n Geo']
['aerologi', '', '', '', 'n', '', 'n Fis']
['aerometer', '', '', '', 'n', '', 'n Kim']
['aerometer', '', '', '', 'n', '', '']
['aeronautika', '', '', '', 'n', '', '']
['aeronom', '', '', '', 'n', '', '']
['aeronomi', '', '', '', 'n', '', '']
['aeroplangton', '', '', '', 'n', '', 'n Biol']
['aeroskop', '', '', '', 'n', '', '']
['aerosol', '', '', '', 'n', '', 'n Kim']
['aerotonometer', '', '', '', 'n', '', '']
['afair', '', '', '', 'n', '', '']
['afal', '', '', '', 'n', '', '']
['afasia', '', '', '', 'n', '', 'n Psi']
['afdal', '', '', '', 'n', '', '']
['afdol', '', '', '', '', 'afdal', '']
['afek', '', '', '', 'n', '', 'n Psi Dok']
['afeksi', '', '', '', 'n', '', 'n Psi']
['afektif', '', '', '', 'a', '', 'a Psi']
['', '', '', '', '', '', 'Dok']
['aferesis', '', '', '', 'n

['', '', 'air perak', '', '', '', '']
['', '', 'air raksa', '', '', '', '']
['', '', 'air sadah', '', '', '', '']
['', '', 'air sebak', '', '', '', '']
['', '', 'air sembahyang', '', '', '', '']
['', '', 'air sembilan', '', '', '', '']
['', '', 'air seni', '', '', '', '']
['', '', 'air senyawa', '', '', '', '']
['', '', 'air serani', '', '', '', 'Kris']
['', '', 'air setaman', '', '', '', '']
['', '', 'air soda', '', '', '', '']
['', '', 'air suci', '', '', '', 'Kris']
['', '', 'air suling', '', '', '', '']
['', '', 'air suri', '', '', '', '']
['', '', 'air surut', '', '', '', '']
['', '', 'air susu', '', '', '', '']
['', '', 'air talkin', '', '', '', '']
['', '', 'air tanah', '', '', '', 'Geog']
['', '', 'air tawar', '', '', '', '']
['', '', 'air tepung', '', '', '', '']
['', '', 'air terjun', '', '', '', '']
['', '', 'air timah', '', '', '', '']
['', '', 'air timpas', '', '', '', '']
['', '', 'air tolak', '', '', '', '']
['', '', 'air tuban', '', '', '', '']
['', '', 'air utama', '',

['akropolis', '', '', '', 'n', '', '']
['akrosentrik', '', '', '', 'n', '', '']
['aksa', '', '', '', 'a', '', '']
['aksara', '', '', '', 'n', '', '']
['', 'bukan-numerik', '', '', '', '', '']
['', '', 'aksara periksa', '', '', '', '']
['akselerasi', '', '', '', 'n', '', '']
['', '', '', '', '', '', 'Fis']
['akselerator', '', '', '', 'n', '', '']
['akselerometer', '', '', '', 'n', '', 'n Kim']
['aksen', '', '', '', 'n', '', '']
['aksen', '', '', '', 'n', '', '']
['aksentuasi', '', '', '', 'n', '', '']
['aksep', '', '', '', 'n', '', '']
['', '', '', '', '', '', 'Ek']
['', 'mengaksep', '', '', 'v', '', '']
['akseptabel', '', '', '', 'n', '', '']
['akseptasi', '', '', '', 'n', '', '']
['akseptor', '', '', '', 'n', '', 'n Dag']
['akserator', '', '', '', 'n', '', '']
['aksesori', '', '', '', 'n', '', '']
['aksi', '', '', '', 'n', '', '']
['', '', 'aksi polisional', '', '', '', '']
['', '', 'aksi sosial', '', '', '', '']
['', 'beraksi', '', '', 'v', '', '']
['aksioma', '', '', '', 'n', '', ''

...

['', '', 'perusahaan dagang', '', '', '', '']
['', '', 'perusahaan ekspedisi', '', '', '', '']
['', '', 'perusahaan induk', '', '', '', '']
['', '', 'perusahaan minyak', '', '', '', '']
['', '', 'perusahaan negara', '', '', '', '']
['', '', 'perusahaan perkapalan', '', '', '', '']
['', 'pengusaha', '', '', 'n', '', '']
['', '', 'pengusaha kecil', '', '', '', 'Ek']
['', '', 'pengusaha menengah', '', '', '', 'Ek']
['', 'pengusahaan', '', '', 'n', '', '']
['', 'kepengusahaan', '', '', 'n', '', '']
['usahawan', '', '', '', 'n', '', '']
['', 'keusahawanan', '', '', 'n', '', '']
['usai', '', '', '', 'v', '', '']
['', 'mengusai', '', '', 'v', '', '']
['usak', '', '', '', 'v', '', '']
['', 'mengusaki', '', '', 'v', '', '']
['usali', '', '', '', 'n', '', '']
['', 'berusali', '', '', 'v', '', '']
['usam', '', '', '', 'n', '', '']
['usam', '', '', '', 'a', '', 'ark a']
['usang', '', '', '', 'a', '', '']
['', 'mengusang', '', '', 'v', '', '']
['', 'keusangan', '', '', 'n', '', '']
['usap', '', '',

['virogen', '', '', '', 'n', '', 'n Bio']
['virologi', '', '', '', 'n', '', '']
['virosis', '', '', '', 'n', '', 'n Dok']
['virtual', '', '', '', 'a', '', '']
['virtuoso', '', '', '', 'n', '', '']
['virulen', '', '', '', 'a', '', 'a Dok']
['virulensi', '', '', '', 'n', '', 'n Dok']
['virus', '', '', '', 'n', '', '']
['', '', 'virus desrues', '', '', '', '']
['', '', 'virus enteritis', '', '', '', '']
['', '', 'virus fiksa', '', '', '', '']
['', '', 'virus flu burung', '', '', '', '']
['', '', 'virus hepatitis infeksiosa', '', '', '', '']
['', '', 'virus mosaik', '', '', '', 'Kim']
['', '', 'virus vaccinicum', '', '', '', '']
['virusidal', '', '', '', 'a', '', '']
['visa', '', '', '', 'n', '', '']
['visi', '', '', '', 'n', '', '']
['visibel', '', '', '', 'a', '', '']
['visibilitas', '', '', '', 'n', '', '']
['visiun', '', '', '', 'n', '', '']
['viskometer', '', '', '', '', '', '']
['viskose', '', '', '', 'n', '', '']
['viskositas', '', '', '', 'n', '', '']
['vista', '', '', '', 'n', '',

['wangi-wangian', '', '', '', 'n', '', '']
['wewangian', '', '', '', 'n', '', '']
['', 'mewangi', '', '', 'v', '', '']
['', 'mewangikan', '', '', 'v', '', '']
['', 'pewangi', '', '', 'n', '', '']
['', 'kewangian', '', '', 'a', '', '']
['', '', '', '', 'n', '', '']
['', 'kewangi-wangian', '', '', 'a', '', '']
['wangkang', '', '', '', 'n', '', '']
['wangsa', '', '', '', 'n', '', 'kl n']
['wangsit', '', '', '', 'n', '', '']
['wani', '', '', '', 'a', '', '']
['wanita', '', '', '', 'n', '', '']
['', '', 'wanita karier', '', '', '', '']
['', 'kewanitaan', '', '', 'n', '', '']
['wanoja', '', '', '', 'n', '', '']
['wanprestasi', '', '', '', 'a', '', '']
['', '', '', '', 'n', '', '']
['wantek', '', '', '', 'n', '', '']
['', 'mewantek', '', '', 'v', '', '']
['', 'pewantek', '', '', 'n', '', '']
['wantilan', '', '', '', 'n', '', '']
['wantilan', '', '', '', 'n', '', '']
['Wara', '', '', '', 'n', '', 'n akr']
['wara', '', '', '', '', '', '']
['', 'wara-wara', '', '', 'n', '', 'kl n']
['', 'pewara'

['wenter', '', '', '', 'n', '', '']
['wereng', '', '', '', 'n', '', '']
['', '', 'wereng cokelat', '', '', '', '']
['', '', 'wereng hijau', '', '', '', '']
['werst', '', '', '', 'n', '', '']
['wese', '', '', '', 'n', '', 'n kp']
['wesel', '', '', '', 'n', '', '']
['', '', 'wesel bayar', '', '', '', '']
['', '', 'wesel bank', '', '', '', '']
['', '', 'wesel berjangka', '', '', '', '']
['', '', 'wesel cek', '', '', '', '']
['', '', 'wesel luar', '', '', '', '']
['', '', 'wesel permintaan', '', '', '', '']
['', 'terjamin', 'wesel tagih', '', '', '', '']
['', '', 'wesel tunda', '', '', '', '']
['wesel', '', '', '', 'n', '', '']
['wesket', '', '', '', 'n', '', 'ark n']
['westernis', '', '', '', 'a', '', '']
['westernisasi', '', '', '', 'n', '', '']
['wet', '', '', '', 'n', '', 'kl n']
['wetan', '', '', '', 'n', '', '']
['weton', '', '', '', 'n', '', '']
['wewarah', '', '', '', 'n', '', '']
['wewaton', '', '', '', 'n', '', '']
['wibawa', '', '', '', 'n', '', '']
['', 'berwibawa', '', '', 'v'

['yuwana', '', '', '', 'a', '', '']
['', '', '', '', 'n', '', '']
['yuyitsu', '', '', '', 'n', '', '']
['yuyu', '', '', '', 'n', '', '']
['Z', '', '', '', '', '', '']
['z', '', '', '', 'n', '', '']
['', '', '', '', '', 'jebat', '']
['zabah', '', '', '', 'v', '', '']
['zabaniah', '', '', '', 'n', '', '']
['zabarjad', '', '', '', 'n', '', '']
['zabib', '', '', '', 'n', '', '']
['Zabur', '', '', '', 'n', '', '']
['zadat', '', '', '', 'n', '', 'n Fis kp']
['zahid', '', '', '', 'n', '', '']
['', '', '', '', '', 'lahir', '']
['zair', '', '', '', 'n', '', '']
['zaitun', '', '', '', 'n', '', '']
['zakar', '', '', '', 'n', '', '']
['Zakaria', '', '', '', 'n', '', '']
['zakat', '', '', '', 'n', '', '']
['', '', 'zakat fitrah', '', '', '', '']
['', '', 'zakat harta', '', '', '', '']
['', '', 'zakat mal', '', '', '', '']
['', 'berzakat', '', '', 'v', '', '']
['zakiah', '', '', '', 'a', '', '']
['zakirah', '', '', '', 'a', '', '']
['zakum', '', '', '', 'n', '', '']
['', '', '', '', '', 'lalim', '']